In [172]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import tensorflow as tf
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error

%matplotlib inline

from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10, 6

In [159]:
dataset = pd.read_csv(r"AirQualityUCI.csv", sep=";", parse_dates={'datetime':[0,1]}).drop(columns=["Unnamed: 15","Unnamed: 16"],axis=1).dropna()

In [160]:
columns = ['CO(GT)', 'C6H6(GT)', 'T', 'RH', 'AH']
for col in columns:
    dataset[col] = dataset[col].apply(lambda x: float(str(x).replace(",",'.')) )

In [161]:
dataset['datetime'] = pd.to_datetime(dataset['datetime'],format='%d/%m/%Y %H.%M.%S')

In [162]:
dataset.tail()

,datetime,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
9352,2005-04-04 10:00:00,3.1,1314.0,-200.0,13.5,1101.0,472.0,539.0,190.0,1374.0,1729.0,21.9,29.3,0.7568
9353,2005-04-04 11:00:00,2.4,1163.0,-200.0,11.4,1027.0,353.0,604.0,179.0,1264.0,1269.0,24.3,23.7,0.7119
9354,2005-04-04 12:00:00,2.4,1142.0,-200.0,12.4,1063.0,293.0,603.0,175.0,1241.0,1092.0,26.9,18.3,0.6406
9355,2005-04-04 13:00:00,2.1,1003.0,-200.0,9.5,961.0,235.0,702.0,156.0,1041.0,770.0,28.3,13.5,0.5139
9356,2005-04-04 14:00:00,2.2,1071.0,-200.0,11.9,1047.0,265.0,654.0,168.0,1129.0,816.0,28.5,13.1,0.5028


In [163]:
#dataset.dtypes

In [164]:
#dataset.isnull().sum()

In [165]:
data = dataset.drop(['datetime'], axis=1)
data.index = dataset.datetime
#checking stationarity
from statsmodels.tsa.vector_ar.vecm import coint_johansen
#since the test works for only 12 variables, I have randomly dropped
#in the next iteration, I would drop another and check the eigenvalues
johan_test_temp = data.drop(['CO(GT)'], axis=1)
#johan_test_temp = data
coint_johansen(johan_test_temp,-1,1).eig

array([1.85137431e-01, 1.49562379e-01, 9.98466990e-02, 9.15622257e-02,
       6.59920501e-02, 5.02055763e-02, 4.01484678e-02, 3.35580944e-02,
       1.84178447e-02, 1.50158406e-02, 4.14847921e-03, 7.21944108e-05])

In [166]:
#creating the train and validation set
train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

cols = data.columns

for j in cols:
    for i in range(0,len(data)):
       if data[j][i] == -200:
           data[j][i] = data[j][i-1]

#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

/opt/conda/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
/opt/conda/lib/python3.8/site-packages/statsmodels/base/wrapper.py:34: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


In [173]:
#converting predictions to dataframe
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,13):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

#check rmse
j = 0
for i in cols:
    print('rmse value for', i, 'is : ', math.sqrt(mean_absolute_error(pred.iloc[:,j], valid.iloc[:,j])))
    j +=1

rmse value for CO(GT) is :  1.066462254187852
rmse value for PT08.S1(CO) is :  13.001824321416066
rmse value for NMHC(GT) is :  2.5302971314862863
rmse value for C6H6(GT) is :  2.4820888653216437
rmse value for PT08.S2(NMHC) is :  15.422996982837953
rmse value for NOx(GT) is :  12.329249935169996
rmse value for PT08.S3(NOx) is :  13.958608690249868
rmse value for NO2(GT) is :  7.259374277148832
rmse value for PT08.S4(NO2) is :  20.86420786014703
rmse value for PT08.S5(O3) is :  19.235469130809918
rmse value for T is :  3.0662985181056874
rmse value for RH is :  3.7422152961864836
rmse value for AH is :  0.6793153728684356
